In [9]:
require 'poisson'

class Modeling
  def initialize(people_num, possibility_correct)
    @people_num = people_num
    @people_half = half_num(@people_num) # 終了の値や半分の値に用いる
    @possibility_correct = possibility_correct
  end

  # ポアソン分布のテスト
  def test_random
    poisson = Poisson.new(50) # 平均50のポアソン分布
    sum = 0
    40.times do |t|
      sum += poisson.probability{ |x| x == t + 30 }
    end
    p sum
  end

  # 一人目の意見を採用するという雑魚アルゴリズム
  # TODO 時刻0から99で行なっているが1から100の方が好ましい気がする
  def baseline_method_deciding_by_first_person_with_poisson(possibility_correct)
    collecting_deadline = 60 # 意見収集の締め切り 59時刻
    poisson = Poisson.new(30) # 平均30人のポアソン分布
    average_method_utility = 0
    # 0~59人来るときのみを考慮する(それ以外は0に近似する)
    60.times do |t_people|
      # t_people人来るとき
      when_people_come = Array.new(collecting_deadline) # 人がいつ来るか配列で表す
      temp_probability_by_poisson = poisson.probability{ |x| x == t_people }
      t_people.times do |t|
        when_people_come[t] = 1
      end
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ないときは飛ばす
        weight = 0.6.to_f / collecting_deadline # 時間がかかることに対する重み
        method_utility += possibility_correct - index.to_f * weight
        break
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 5人で多数決して意見集約を行うというアルゴリズム
  def baseline_method_deciding_by_majority_vote_with_poisson(possibility_correct)
    collecting_deadline = 60 # 意見収集の締め切り 59時刻
    poisson = Poisson.new(30) # 平均30人のポアソン分布
    average_method_utility = 0
    # 0~59人来るときのみを考慮する(それ以外は0に近似する)
    60.times do |t_people|
      # t_people人来るとき
      when_people_come = Array.new(collecting_deadline) # 人がいつ来るか配列で表す
      temp_probability_by_poisson = poisson.probability{ |x| x == t_people }
      t_people.times do |t|
        when_people_come[t] = 1
      end
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      next if t_people < 5 # 5人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
        if people_count == 5
          weight = 0.6.to_f / collecting_deadline
          method_utility +=  (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - index.to_f * weight
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  def half_num(num)
    (num.to_f / 2).ceil
  end

  # nCk 実行例 5C2 = 5 * 4 / 2! = 10
  def combi(n, k)
    k = n - k if 2 * k > n
    return 1 if k == 0
    ((n - k + 1)..n).reduce(&:*) / (1..k).reduce(&:*)
  end

  # 誤差率をもとめる.先にある割合の意見(finish_num)を集めた方を意見集約の結果とするアルゴリズム
  # このアルゴリズムは必要な人数が動的に変化するので、誤差率をfinish_num個の要素にした配列で返す
  def relative_error_array_by_half_opinion(finish_num, possibility_correct) # possibility_correct => 人が正解する確率p
    relative_error = []
    finish_num.times do |t|
      relative_error << ((1 - possibility_correct)**finish_num) * (possibility_correct**t) * combi(finish_num - 1 + t, t)
    end
    relative_error
  end

  # 誤差率をもとめる.people_num人で多数決を行い意見集約の結果とするアルゴリズム
  def relative_error_by_majority_vote(people_num, possibility_correct) # possibility_correct => 人が正解する確率p
    half_num = half_num(people_num)
    relative_error = 0
    half_num.times do |t|
      relative_error += (possibility_correct**t) * ((1 - possibility_correct)**(people_num - t)) * combi(people_num, t)
    end
    relative_error
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method1(possibility_correct) # possibility_correct => 人が正解する確率p
    # 終了する人数を求める
    finish_num = @people_half
    # 誤差率を求め、返す
    relative_error_array_by_half_opinion(finish_num, possibility_correct).inject(:+)
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method2(possibility_correct)
    # 多数決を行い誤差率を求め、返す
    relative_error_by_majority_vote(@people_num, possibility_correct)
  end

  # 誤差率εを変動させて、必要な人数Xを求める
  def baseline_method3(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 終了する人数を求める
    temp_finish_num = half_num(people_num)
    # 必要な平均人数
    average_num = 0
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      relative_error_array = relative_error_array_by_half_opinion(temp_finish_num, @possibility_correct)
      expected_error = relative_error_array.inject(:+)
      average_num = 0
      relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
        average_num += e * (i + temp_finish_num) / expected_error
      end
      break if expected_error <= relative_error
      people_num += 2
      temp_finish_num = half_num(people_num)
    end
    # 必要な人数の平均を返す
    average_num
  end

  # 多数決を意見集約のアルゴリズムとする
  def baseline_method4(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      expected_error = relative_error_by_majority_vote(people_num, @possibility_correct)
      break if expected_error <= relative_error
      people_num += 2
    end
    # 必要な人数を返す
    people_num
  end
end


require 'rbplotly'

PEOPELE_NUM = 199
POSSIBILITY_CORRECT = 0.7
DELIMITER = 100

model = Modeling.new(PEOPELE_NUM, POSSIBILITY_CORRECT)

# p model.baseline_method_deciding_by_majority_vote_with_poisson

# 縦軸 => 効用utility, 横軸 => 人の正解する確率p
utility_possibility_x_axis = (DELIMITER / 2..DELIMITER).to_a
utility_possibility_by_first_person_traces_arr = []
10.times do
  utility_possibility_by_first_person_traces_arr << { x: utility_possibility_x_axis, y: utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_first_person_with_poisson(e.to_f / DELIMITER) } }
end
pl_by_first_person = Plotly::Plot.new(data: utility_possibility_by_first_person_traces_arr)
pl_by_first_person.layout.xaxis = { title: 'possibility_correct' }
pl_by_first_person.layout.yaxis = { title: 'utility' }
pl_by_first_person.show

utility_possibility_by_majority_vote_traces_arr = []
10.times do
  utility_possibility_by_majority_vote_traces_arr << { x: utility_possibility_x_axis, y: utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_majority_vote_with_poisson(e.to_f / DELIMITER) } }
end
pl_by_majority_vote = Plotly::Plot.new(data: utility_possibility_by_majority_vote_traces_arr)
pl_by_majority_vote.layout.xaxis = { title: 'possibility_correct' }
pl_by_majority_vote.layout.yaxis = { title: 'utility' }
pl_by_majority_vote.show





(pry):1746: warning: already initialized constant PEOPELE_NUM
(pry):1554: warning: previous definition of PEOPELE_NUM was here
(pry):1747: warning: already initialized constant POSSIBILITY_CORRECT
(pry):1555: warning: previous definition of POSSIBILITY_CORRECT was here
(pry):1748: warning: already initialized constant DELIMITER
(pry):1556: warning: previous definition of DELIMITER was here


#<Plotly::Offline::HTML:0x00007fb02c8e3378 @id="02e5bec4-baca-4317-9d9b-1c9d35085625", @data=[{:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.4904254828571503, 0.5023185219199937, 0.5111547360416698, 0.5200450007731655, 0.5297662866852557, 0.5327291120966758, 0.5473295180109053, 0.5577602746097515, 0.5702382084451922, 0.5818009926198964, 0.5918216692262847, 0.6031462343898478, 0.6105177648314958, 0.6222309475388489, 0.6312030128360455, 0.6417429472050098, 0.6522309102347426, 0.6590836949742892, 0.6687145484330691, 0.6765049551516872, 0.694186479627947, 0.6989658607051743, 0.7090163618355699, 0.7154012481690053, 0.7313043149554382, 0.7468095796941779, 0.7444726785926754, 0.7478956023344507, 0.7684750519614969, 0.776573970022511, 0.7869610563287301, 0.79943065128941, 0.8089880410500777, 0.8199735582157671, 0.8331958960813702, 0.8436711513971644, 0.8520604760370697, 0.8553601317554727, 0.8628355283456357, 0.873168342032951, 0.8920365443499562, 0.8969147016740294, 0.9083978699916111, 0.9209266969505384, 0.9267023589344915, 0.9392157144822938, 0.9465512334602698, 0.9617611270292661, 0.9687881971073012, 0.9738369976754633, 0.990272295818003]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.48821772408867514, 0.5019206749871695, 0.501867452747178, 0.5204147821895807, 0.5251180688625886, 0.5389085074610355, 0.5446114117263221, 0.5620953461036194, 0.5702958921420358, 0.5793767161446168, 0.5888365512516012, 0.5996748718112753, 0.608764542110355, 0.6210351120055939, 0.6239220940886036, 0.6372638380101027, 0.643596827268761, 0.6566546177223277, 0.6677725043989727, 0.6803922959643379, 0.6916151684942637, 0.7033321971024189, 0.7121327051496508, 0.7184670969427124, 0.72494471349215, 0.7414563425855384, 0.7453601690627774, 0.7613066738380905, 0.7681988818489941, 0.7800326595108323, 0.791469045493129, 0.8008702728739892, 0.8117283187075073, 0.8219786722773368, 0.8348765999432651, 0.8370465881082153, 0.8474918519753134, 0.8621088889412064, 0.873540193807282, 0.8750127600601448, 0.8915970397577819, 0.9019741925930375, 0.9057618864152984, 0.9182330982074723, 0.9266117211722739, 0.9391253362905589, 0.948932962154981, 0.9620195090832571, 0.9676485328616278, 0.9803803588779901, 0.9882514316020317]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.49046325390188106, 0.5040586110745588, 0.5110196729790707, 0.5217485615418835, 0.5243611300589377, 0.5416708742890466, 0.5482943187622832, 0.5604452821243302, 0.5667343067132748, 0.5843150621520904, 0.5898665797832139, 0.5953252197565607, 0.6068031998177534, 0.6158445249805706, 0.6317804502105248, 0.6394249055881588, 0.6472435034464669, 0.6586779772269079, 0.6722214503442427, 0.6820055969018982, 0.6823292969474856, 0.6948992589174248, 0.7115475416311814, 0.7190299460633219, 0.7308049568969389, 0.7393324769044749, 0.7546379829453327, 0.7622959103775488, 0.7625597797395325, 0.7762916647266663, 0.7858642348808714, 0.8009348226713698, 0.8025597920808054, 0.822471325905224, 0.8226461680380116, 0.8418429254174822, 0.8492057587663295, 0.8638971171104625, 0.8718860616593821, 0.8803302514995273, 0.8921338665052932, 0.8981432059744611, 0.9072397538400826, 0.9204149032561337, 0.9329643537184396, 0.9398792149266857, 0.9485064072101204, 0.9585789453795299, 0.9722495283894852, 0.9829621279633477, 0.9909443178956048]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 

#<Plotly::Offline::HTML:0x00007fb02d3b8b90 @id="9eac5422-57a2-4dcc-b8ac-64c56fc5c033", @data=[{:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.41401849845859806, 0.4104550929372914, 0.44618827172460146, 0.47432842448851065, 0.489894515447235, 0.5036139519162403, 0.5241393521570556, 0.5350440907240709, 0.5567352617620417, 0.5803402101718481, 0.5787981416840764, 0.607460634592525, 0.6176449262186023, 0.6408933254162088, 0.6630193650907031, 0.675034622793744, 0.6854266865713784, 0.699392542058715, 0.7100806128516609, 0.7466077164324069, 0.7380258309629224, 0.7555253403546454, 0.7662251028870897, 0.784038565118566, 0.7930468407734189, 0.8003058427124263, 0.8072589198380433, 0.8229798002680159, 0.8271931089192733, 0.840839001226073, 0.8469377444159022, 0.8687677891020923, 0.8668126108988871, 0.86857993551998, 0.8750148605868155, 0.891578948849751, 0.8830101192000638, 0.8880950761173693, 0.8831279147927578, 0.892355450464318, 0.8895297262845566, 0.8989316641871171, 0.9016482910569638, 0.9061702038160324, 0.8964219991345898, 0.9037468401897318, 0.9216264537681206, 0.905630783689708, 0.9011166406575462, 0.91601336613902, 0.9111284317684818]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.4158759484152857, 0.41931792609881485, 0.4572222719718339, 0.4743113251801879, 0.4901184669590915, 0.5008545798332449, 0.5171041743108578, 0.5507217383587326, 0.5539634099811095, 0.5780865064640498, 0.5912993347772061, 0.6066158065742938, 0.6164026716014072, 0.6431784903244926, 0.6574169380416572, 0.6706164710699878, 0.687240256222549, 0.7010575448183543, 0.7161353434571588, 0.7392668044490437, 0.748196794311743, 0.7496993653946344, 0.767182090300729, 0.7901366763009413, 0.8054916252059378, 0.8062327513081293, 0.8275560745407486, 0.8152934154966822, 0.8268892738721753, 0.8263086561664532, 0.850449815031249, 0.8749373305611333, 0.8601754205712421, 0.870042952505438, 0.8805351212258298, 0.8875472875369254, 0.8830454185315311, 0.8880994111206738, 0.9060322015068165, 0.9034218692819282, 0.8973679385626696, 0.9136327102132187, 0.9006603687197653, 0.9055629186511133, 0.9034323212528498, 0.9072602403429633, 0.8993709070458268, 0.9041007786416714, 0.9118907139329883, 0.9091402337333935, 0.914379260963871]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.390689809174858, 0.41907648226625555, 0.44222292022053195, 0.4677034978231181, 0.47446602894927137, 0.49858086279169966, 0.5258875551361435, 0.5379556624107106, 0.550035796797428, 0.5804329629312223, 0.5866701963083776, 0.6166072144077493, 0.6291406687830202, 0.6456687244972378, 0.6583604361359382, 0.6629030554703489, 0.6883025224816679, 0.6963584611747399, 0.7198069369359629, 0.7172748236534029, 0.7435328579336349, 0.751419501922982, 0.769902452126536, 0.7870112570463662, 0.8023617913274279, 0.8086032752364761, 0.8114590727140427, 0.8296217408976467, 0.821139431388738, 0.8320518935425222, 0.8530100385838733, 0.8548860337624131, 0.861394357808723, 0.8718737494823879, 0.8858818245921426, 0.8778355316910393, 0.8836195550951469, 0.892198453840704, 0.9090461729139152, 0.8933492212130856, 0.9109018023267129, 0.8948125582553395, 0.9021501711272433, 0.9001494780076428, 0.9291262378715068, 0.9197538652023051, 0.9071848981637703, 0.913529965268532, 0.9083599209929082, 0.9135307293391739, 0.9047434373093624]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 10

#<CZTop::Socket::PUB:0x7fb02e5cc840 last_endpoint="tcp://127.0.0.1:55365">